In [44]:
import cv2
import numpy as np
import os

In [45]:
# def extract_sudoku_grid(image_path):
#     # Step 1: Read the image and preprocess
#     img = cv2.imread(image_path)
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     blurred = cv2.GaussianBlur(gray, (5, 5), 0)
#     edges = cv2.Canny(blurred, 50, 150)
#     cv2.imshow("edges", edges)
#     cv2.waitKey(0)
#     # Step 2: Detect contours and find the largest quadrilateral
#     contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     largest_contour = max(contours, key=cv2.contourArea)
#     img_contour_before = img.copy()
#     cv2.drawContours(img_contour_before, [largest_contour], -1, (255, 0, 0), 3)
#     cv2.imshow("Largest Contour Before Approximation", img_contour_before)
#     cv2.waitKey(0)
#     approx = cv2.approxPolyDP(largest_contour, 0.05 * cv2.arcLength(largest_contour, True), True)
#     img_contour = img.copy()
#     cv2.drawContours(img_contour, [approx], -1, (0, 255, 0), 3)
#     cv2.imshow("contour", img_contour)
#     cv2.waitKey(0)
#     # Step 3: Apply perspective transformation
#     if len(approx) == 4:  # Ensure it's a quadrilateral
#         pts = np.float32([point[0] for point in approx])  # Extract points
#         # Sort points (top-left, top-right, bottom-right, bottom-left)
#         rect = order_points(pts)
#         # Apply perspective transformation
#         transformed = warp_perspective(img, rect)
#         return transformed
#     else:
#         print("No quadrilateral detected")
#         return None
# 
# def order_points(pts):
#     # Order points to [top-left, top-right, bottom-right, bottom-left]
#     rect = np.zeros((4, 2), dtype="float32")
#     s = pts.sum(axis=1)
#     diff = np.diff(pts, axis=1)
#     rect[0] = pts[np.argmin(s)]  # Top-left
#     rect[2] = pts[np.argmax(s)]  # Bottom-right
#     rect[1] = pts[np.argmin(diff)]  # Top-right
#     rect[3] = pts[np.argmax(diff)]  # Bottom-left
#     return rect
# 
# def warp_perspective(img, rect):
#     # Define the new image dimensions (e.g., 450x450 pixels)
#     size = 450
#     dst = np.array([[0, 0], [size - 1, 0], [size - 1, size - 1], [0, size - 1]], dtype="float32")
#     # Get the transformation matrix and apply it
#     M = cv2.getPerspectiveTransform(rect, dst)
#     warped = cv2.warpPerspective(img, M, (size, size))
#     return warped

In [46]:
# sudoku_grid = extract_sudoku_grid(image_path)
# cv2.imshow("Sudoku Grid", sudoku_grid)
# cv2.waitKey(0)

In [47]:
# def filter_candidates_with_hough(image, square_candidates, min_lines=50):
#     """
#     Filter square candidates by detecting many straight lines (likely Sudoku grids).
#     
#     Args:
#         image: The original image (grayscale or edges) for Hough Line Transform.
#         square_candidates: List of contours representing square candidates.
#         min_lines: Minimum number of lines required to consider a candidate as valid.
# 
#     Returns:
#         List of contours that likely represent Sudoku grids.
#     """
#     valid_sudoku_candidates = []
# 
#     for candidate in square_candidates:
#         # Create a mask for the candidate
#         mask = np.zeros_like(image)
#         cv2.drawContours(mask, [candidate], -1, 255, thickness=cv2.FILLED)
# 
#         # Extract the region of interest (ROI)
#         roi = cv2.bitwise_and(image, image, mask=mask)
# 
#         # Apply Hough Line Transform
#         lines = cv2.HoughLines(cv2.Canny(roi, 50, 150), 1, np.pi / 180, threshold=100)
# 
#         # Debug visualization (optional, draw lines on ROI)
#         if lines is not None:
#             roi_with_lines = cv2.cvtColor(roi, cv2.COLOR_GRAY2BGR)
#             for rho, theta in lines[:, 0]:
#                 a = np.cos(theta)
#                 b = np.sin(theta)
#                 x0 = a * rho
#                 y0 = b * rho
#                 x1 = int(x0 + 1000 * (-b))
#                 y1 = int(y0 + 1000 * (a))
#                 x2 = int(x0 - 1000 * (-b))
#                 y2 = int(y0 - 1000 * (a))
#                 cv2.line(roi_with_lines, (x1, y1), (x2, y2), (0, 255, 0), 1)
# 
#             # Optional: visualize ROI with detected lines
#             cv2.imshow("ROI with Lines", roi_with_lines)
#             cv2.waitKey(1)
# 
#         # Count lines and filter
#         line_count = len(lines) if lines is not None else 0
#         if line_count >= min_lines:
#             print(f"Detected {line_count} lines for a valid candidate.")
#             valid_sudoku_candidates.append(candidate)
#     
#     cv2.waitKey(0)
#     # Cleanup visualization
#     cv2.destroyAllWindows()
#     return valid_sudoku_candidates
# def process_contours(contours, aspect_ratio_epsilon=0.15, binary_search_epsilon=1e-5):
#     """
#     Process contours to identify valid square-like quadrilaterals.
#     
#     Args:
#         contours: List of contours to process.
#         aspect_ratio_epsilon: Allowed deviation for aspect ratio to consider as square.
#         binary_search_epsilon: Tolerance for binary search termination in polygon approximation.
# 
#     Returns:
#         List of contours that are valid squares or quadrilaterals.
#     """
#     valid_squares = []
# 
#     for contour in contours:
#         perimeter = cv2.arcLength(contour, True)
#         low, high = 0, 0.5  # Epsilon range (0 to 50% of the perimeter)
# 
#         while high - low > binary_search_epsilon:
#             epsilon = (low + high) / 2
#             approx = cv2.approxPolyDP(contour, epsilon * perimeter, True)
# 
#             # Debug visualization (optional, to see intermediate approximations)
#             #draw_all_contours([approx], title="Approximation")
# 
#             # Check if the approximation is a quadrilateral
#             if len(approx) == 4:
#                 # Validate aspect ratio
#                 x, y, w, h = cv2.boundingRect(approx)
#                 aspect_ratio = w / float(h)
#                 if abs(aspect_ratio - 1) < aspect_ratio_epsilon:
#                     valid_squares.append(approx)  # Found a valid square
#                     break  # Move to the next contour
#                 else:
#                     # Quadrilateral failed aspect ratio test; refine further
#                     high = epsilon
#             elif len(approx) > 4:
#                 # Too many sides, increase epsilon to simplify
#                 low = epsilon
#             else:
#                 # Too few sides, decrease epsilon to capture more detail
#                 high = epsilon
# 
#     return valid_squares
# 
# def detect_sudoku_grid(image):
#     # Step 1: Preprocess the image
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     blurred = cv2.GaussianBlur(gray, (5, 5), 0)
#     edges = cv2.Canny(blurred, 50, 150)
#     # Step 2: Find contours
#     contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     draw_all_contours(contours, title="All Contours")
#     # Step 3: Filter by aspect ratio
#     square_candidates = process_contours(contours)
#     draw_all_contours(square_candidates, title="Square Candidates")
#     # Step 4: Filter by Hough lines
#     valid_candidates = filter_candidates_with_hough(edges, square_candidates)
#     draw_all_contours(valid_candidates, title="Valid Candidates")
#     return valid_candidates
# def order_points(pts):
#     # Order points to [top-left, top-right, bottom-right, bottom-left]
#     rect = np.zeros((4, 2), dtype="float32")
#     s = pts.sum(axis=1)
#     diff = np.diff(pts, axis=1)
#     rect[0] = pts[np.argmin(s)]  # Top-left
#     rect[2] = pts[np.argmax(s)]  # Bottom-right
#     rect[1] = pts[np.argmin(diff)]  # Top-right
#     rect[3] = pts[np.argmax(diff)]  # Bottom-left
#     return rect
# def warp_perspective(img, rect):
#     # Define the new image dimensions (e.g., 450x450 pixels)
#     size = 450
#     dst = np.array([[0, 0], [size - 1, 0], [size - 1, size - 1], [0, size - 1]], dtype="float32")
#     # Get the transformation matrix and apply it
#     M = cv2.getPerspectiveTransform(rect, dst)
#     warped = cv2.warpPerspective(img, M, (size, size))
#     return warped
# def draw_all_contours(contours,title="All Contours"):
#     print("Contours array size: ", len(contours))
#     img = cv2.imread(image_path)
#     #draw each contour in a different window
#     for i in range(len(contours)):
#         cv2.drawContours(img, contours, i, (0, 255, 0), 3)
#         cv2.imshow(title, img)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()
#     return
#     

In [48]:
# # Load the image
# image_path = "sudoku.png"
# image = cv2.imread(image_path)
# 
# # Detect the Sudoku grid (quadrilateral points)
# sudoku_points = detect_sudoku_grid(image)
# 
# if sudoku_points is not None:
#     # Order the points (top-left, top-right, bottom-right, bottom-left)
#     print(sudoku_points)
#     rect = order_points(np.array([point[0] for point in sudoku_points]))
# 
#     # Warp the perspective to get a straightened Sudoku grid
#     sudoku_grid = warp_perspective(image, rect)
# 
#     # Display the resulting grid
#     cv2.imshow("Sudoku Grid", sudoku_grid)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
# else:
#     print("No Sudoku grid detected.")


In [49]:
def filter_candidates_with_hough(edges, square_candidates, min_lines=50):
    """
    Filter square candidates by detecting many straight lines (likely Sudoku grids).
    
    Args:
        edges: Edge-detected image (result of Canny or similar).
        square_candidates: List of contours representing square candidates.
        min_lines: Minimum number of lines required to consider a candidate as valid.

    Returns:
        List of contours that likely represent Sudoku grids.
    """
    valid_sudoku_candidates = []

    for candidate in square_candidates:
        # Create a mask for the candidate
        mask = np.zeros_like(edges)
        cv2.drawContours(mask, [candidate], -1, 255, thickness=cv2.FILLED)

        # Extract the region of interest (ROI) using the mask
        roi = cv2.bitwise_and(edges, edges, mask=mask)

        # Apply Hough Line Transform
        lines = cv2.HoughLines(roi, 1, np.pi / 180, threshold=100)

        # Debug visualization (optional)
        if lines is not None:
            roi_with_lines = cv2.cvtColor(roi, cv2.COLOR_GRAY2BGR)
            for rho, theta in lines[:, 0]:
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a * rho
                y0 = b * rho
                x1 = int(x0 + 1000 * (-b))
                y1 = int(y0 + 1000 * (a))
                x2 = int(x0 - 1000 * (-b))
                y2 = int(y0 - 1000 * (a))
                cv2.line(roi_with_lines, (x1, y1), (x2, y2), (0, 255, 0), 1)

            # Optional visualization (for debugging)
            cv2.imshow("ROI with Lines", roi_with_lines)
            cv2.waitKey(1)

        # Count lines and filter
        line_count = len(lines) if lines is not None else 0
        if line_count >= min_lines:
            valid_sudoku_candidates.append(candidate)
        print(f"Detected {line_count} lines for a valid candidate.")
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return valid_sudoku_candidates


def process_contours(contours, aspect_ratio_epsilon=0.15, binary_search_epsilon=1e-5):
    """
    Process contours to identify valid square-like quadrilaterals.
    
    Args:
        contours: List of contours to process.
        aspect_ratio_epsilon: Allowed deviation for aspect ratio to consider as square.
        binary_search_epsilon: Tolerance for binary search termination in polygon approximation.

    Returns:
        List of contours that are valid squares or quadrilaterals.
    """
    valid_squares = []

    for contour in contours:
        perimeter = cv2.arcLength(contour, True)
        low, high = 0, 0.5  # Epsilon range (0 to 50% of the perimeter)

        while high - low > binary_search_epsilon:
            epsilon = (low + high) / 2
            approx = cv2.approxPolyDP(contour, epsilon * perimeter, True)

            # Check if the approximation is a quadrilateral
            if len(approx) == 4:
                # Validate aspect ratio
                x, y, w, h = cv2.boundingRect(approx)
                aspect_ratio = w / float(h)
                if abs(aspect_ratio - 1) < aspect_ratio_epsilon:
                    valid_squares.append(approx)  # Found a valid square
                    break  # Move to the next contour
                else:
                    # Quadrilateral failed aspect ratio test; refine further
                    high = epsilon
            elif len(approx) > 4:
                # Too many sides, increase epsilon to simplify
                low = epsilon
            else:
                # Too few sides, decrease epsilon to capture more detail
                high = epsilon

    return valid_squares


def detect_sudoku_grid(image):
    """
    Detect the Sudoku grid in an image.
    
    Args:
        image: Input image.

    Returns:
        List of valid contours representing Sudoku grids.
    """
    # Step 1: Preprocess the image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)

    # Step 2: Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # draw_all_contours(contours, title="All Contours")

    # Step 3: Filter by aspect ratio and shape
    square_candidates = process_contours(contours)
    # draw_all_contours(square_candidates, title="Square Candidates")

    # Step 4: Filter by Hough lines (lots of grid-like lines indicate a Sudoku grid)
    valid_candidates = filter_candidates_with_hough(edges, square_candidates)
    # draw_all_contours(valid_candidates, title="Valid Candidates")

    return valid_candidates


def order_points(pts):
    """
    Order points in [top-left, top-right, bottom-right, bottom-left] format.
    """
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    diff = np.diff(pts, axis=1)
    rect[0] = pts[np.argmin(s)]  # Top-left
    rect[2] = pts[np.argmax(s)]  # Bottom-right
    rect[1] = pts[np.argmin(diff)]  # Top-right
    rect[3] = pts[np.argmax(diff)]  # Bottom-left
    return rect


def warp_perspective(img, rect):
    """
    Warp the perspective of the image to a 450x450 grid.
    """
    size = 450
    dst = np.array([[0, 0], [size - 1, 0], [size - 1, size - 1], [0, size - 1]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(img, M, (size, size))
    return warped

# Main Script
def main_script(image_path):
    image = cv2.imread(image_path)
    
    # Detect the Sudoku grid
    sudoku_candidates = detect_sudoku_grid(image)
    
    if sudoku_candidates:
        # Use the largest valid candidate
        largest_candidate = max(sudoku_candidates, key=cv2.contourArea)
        rect = order_points(np.squeeze(largest_candidate))
    
        # Warp the perspective to get a straightened Sudoku grid
        sudoku_grid = warp_perspective(image, rect)
    
        # Display the resulting grid
        cv2.imshow("Sudoku Grid", sudoku_grid)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("No Sudoku grid detected, or no valid candidates found for this path:", image_path)
        cv2.waitKey(0)
def process_sudoku_images_in_folder(folder_path):
    """
    Loop through all images in the folder, process each, and detect Sudoku grids.
    
    Args:
        folder_path: Path to the folder containing Sudoku images.
    """
    # List all files in the folder
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('png', 'jpg', 'jpeg', 'bmp'))]

    # Process each image file
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        print(f"Processing image: {image_path}")
        main_script(image_path)

In [50]:
process_sudoku_images_in_folder("small dataset")

Processing image: small dataset\image1.jpg
No Sudoku grid detected, or no valid candidates found for this path: small dataset\image1.jpg
Processing image: small dataset\image100.jpg
Detected 0 lines for a valid candidate.
Detected 0 lines for a valid candidate.
Detected 0 lines for a valid candidate.
Detected 2 lines for a valid candidate.
Detected 0 lines for a valid candidate.
Detected 0 lines for a valid candidate.
No Sudoku grid detected, or no valid candidates found for this path: small dataset\image100.jpg
Processing image: small dataset\image1000.jpg
Detected 0 lines for a valid candidate.
Detected 0 lines for a valid candidate.
Detected 0 lines for a valid candidate.
Detected 0 lines for a valid candidate.
Detected 0 lines for a valid candidate.
Detected 0 lines for a valid candidate.
Detected 0 lines for a valid candidate.
Detected 0 lines for a valid candidate.
Detected 0 lines for a valid candidate.
Detected 0 lines for a valid candidate.
Detected 0 lines for a valid candida